# Welcome

Hi there! Welcome to the first IMG Data Science Hacakthon in 2021! As we step into this new year, OKRs are one of the most frequent topics that have been talked about recently. So before we dive into the problem itself, I would like to walk you through the OKRs for this Hackathon.

## Objective:


Enhance Vanguard competitiveness by increasing awareness of competitor trading activities.

## Key Result -- Modeling Success:

Find insights that can help provide investment ideas, build a data-driven story around your findings and how we can leverage them.


### Metric for modeling success key result: 

We are looking for participants to come up with a(n) model, algorithm or visualizations that takes historical 13F holdings data and generates useful insights about investment trends/themes from other institutional investors.


## Key Result -- Learning Success:

Any approach you take should make intuitive and investment sense. Teach us something new about the data so that it can be replicated on different samples.

### Metric for learning success: 

Documentation of the experimentation process and a story told on your data-driven insights that is easily understood and accepted by the business. Code that can be executed to form/support the insights.

# Important note on the reality of working with a real life data science problem

You may find that this hackathon is unable to be fully solved; or it may highlight an unanswerable question about the quality or interpretation of the data. 

This is a real life data science problem at Vanguard and not a carefully crafted competition question! So things may not go as planned - and that is OK! 

As long as we are iteratively experimenting, working towards the objective and sharing our findings, we are creating value with learning and assisting with the promotion of NWoW principles.



In [ ]:
import pandas as pd
import boto3
import numpy as np

s3 = boto3.client('s3')

# The Problem

Within taxable credit research in FIG (the Fixed Income Group), understanding investment strategies and movements of other investors in the market is key to remaining competitive. 
As an analyst in FIG, one of the goals is understanding the investment strategy and movements of other investors in the market. More specifically, by looking at the holdings of investors who are dissimilar to Vanguard across time, we can find insights on how dissimilar market participants change their holdings to come up with  trends and themes for a specific time period.

For example, imagine you know that Vanguard has a certain amount of holdings in a specific stock X. Then you might want to look at how much of X do other institutional investors that you consider Vanguard's competitors, hold. You might also want to look at whether [smart money](https://www.investopedia.com/terms/s/smart-money.asp#:~:text=Smart%20money%20is%20the%20capital,a%20track%20record%20of%20success) has changed its holdings in X recently, indicating their vision about X has changed. For instance, if a number of investors have significantly decreased their holdings in X recently, then that will be an indicator for you to study. You would need to understand whether there is a trend in reducing the holdings of X and why, to assess whether Vanguard would perform better if you changed the holdings in X too. 

In order to understand other investors’ behaviors, we provide you with the 13F holdings data. Form 13F is a quarterly report that is required to be filed by all institutional investment managers with at least $100 million in assets under management. It discloses their equity holdings and in this hackathon, you will be using them for clustering investors/holdings and identifying trends and deviations. 

So how do we define investors that are dissimilar to Vanguard? In our partnership with Columbia University for their Capstone project, the Columbia students used the 13F holdings data to apply unsupervised clustering in order to find clusters of investors that act similarly to each other. Through this process, the students found 2 major investor clusters: a cluster including Vanguard and those investors that behave similarly to vanguard, and a cluster including investors that behave dissimilarly to Vanguard. The students provided a list of around 50 investors that are different than Vanguard by this clustering mechanism. Their method is imperfect but provides a sensible list of investors that are different than Vanguard and that we consider smart investors. In order to look at the holdings of this list of investors, we will provide you with the list of around 50 investors that are characterized as acting differently than Vanguard. You will work with these investors to find interesting patterns and to assess the changes in their holdings throughout time.


# The Opportunity


As we are looking for insights about other smart investors based on 13F data, this Hackathon is an open ended question. You will be asked to tell a data driven story about the insights you found after the hackathon, which fits the spirit of our hackathon.

The 13F data provides information about investors and their holdings. We want to use this information to understand the investment strategies that some of smart investors deploy and what conclusions can be drawn from those strategies. <b>We are providing <span style="color:red">a focus investor list</span> to participants who want to focus on question #2</b>. The output for this hackathon should be your insights from this data, which can include (but not restricted to): 1. Which investors are dissimilar to Vanguard and what investment insights/strategies can we glean from those investors? 2. Overlaid with performance history, what investment conclusions can be drawn from those investment insights/strategies?

The main output for this hackathon should be your insights from the holdings of the provided list of investors that are dissimilar to Vanguard in their holdings. The main output should include answers to ( but not restricted to) : What can we learn from the investors that are dissimilar to Vanguard? What changes do we see in their holdings? And any visualizations of these insights to tell a story in order to help our investment process.
You can gain insights from the data by clustering, data visualizations,  statistical analysis or any other methods you think might help you generate insights.

Participants are encouraged to add any datasets that might be helpful for generating insights to their analysis. Datasets such as pricing data, news trends, etc.

## For an Additional Challenge

As mentioned above, we have created a list of dissimilar investors by clustering analysis. For participants who are looking for an extra challenge, we invite you to apply your own clustering mechanism to create your clusters of investors similar to Vanguard and those dissimilar to Vanguard. However, keep in mind that the main value of this dataset comes from understanding the holdings of those investors dissimilar to Vanguard. 



# Data Overview
In this hackathon, we will provide you with holdings data, company clustering data and some complimentary data. 

The main dataset for this hackathon is the holdings data, which shows the equity holdings of investors that have at least $100 million in AUM. 

The complimentary data provided are mainly:
 1. Information of investors
 
 2. [Sectors](https://en.wikipedia.org/wiki/The_Refinitiv_Business_Classification) of underlying instruments
 
 3. market data for instruments
 
 4. market data of ETFs for each [economic sectors](https://en.wikipedia.org/wiki/The_Refinitiv_Business_Classification) defined by Refinitiv which you can use as an indicator of the performance for each sector.

We will walk you through each dataset below.

## Holding data

13F Holdings data is from the SEC's form 13F, that is a quarterly report required to be filed by institutional investors that have at least $100 million in assets under management. These filings provide more clarity about the biggest investors in the market, as Congress wanted more transparency about their actions.
This data discloses investors' equity holdings, providing insights into what the smart money is doing in the market.

We can load the holding dataset as shown below:

In [ ]:
holding_obj = s3.get_object(Bucket='ds-hackathon-data', Key='02-2021/holding.csv')
holding_df = pd.read_csv(holding_obj['Body'])

## Details on holding data

In the holding.csv dataset, each line represents information about one investor's holdings for an instrument at a specific time period.

The 13F dataset we provide has the following columns:   
#### Instrument_ID: 
    The ID of the instrument, for example, 8590932301

#### COMNAME: 
    The name of the instrument, for example, Apple Ord Shs

#### LGCYINVESTORID:
    The ID of the investor, for example , 2004260

#### FULLNAME: 
    The name of the investor, for example, The Vanguard Group, Inc.

#### HOLDDATE: 
    Date of the 13F holding report (YYYYMMDD)

#### EFFECTFROMDATE: 
    The starting date from which the data is effective

#### EFFECTTODATE: 
    The date until which the data is effective

#### PCTSHOUTHLD: 
    Number of shares held as a percentage of the shares outstanding. Not adjusted for corporate actions and not consolidated with other filing sources.

#### SHSHLD:
    Shares held at the time of filing

#### SHSHLDVAL: 
    Value of investor's holdings in a particular instrument (not adjusted for any corporate actions)

#### ConsolMktVal: 
    Consolidated market capital of the instrument in millions

In [ ]:
holding_df.head()

## Investors Clustering Results

As mentioned above, in our partnership with Columbia University for their Capstone project, the students used the 13F data and used unsupervised clustering to find 2 major investor clusters: a cluster including Vanguard and those investors that behave similarly to vanguard, and a cluster including investors that behave dissimilarly to Vanguard. 

In the dataset below, we provide the results of their clustering analysis. The investors ( identified by the LGCYINVESTORID across all datasets) are clustered into cluster 0 and cluster 1.

The corresponding cluster for each investor can be found on the column called kmeans_2. If  for an investor the column kmeans_2 = 0, that corresponds to Cluster 0 which is the investor cluster including Vanguard and those investors that behave similarly to vanguard.

If for an investor the column kmeans_2 = 1, that corresponds to cluster 1 which is the cluster of investors that behave differently than Vanguard. The second cluster is our focus for this hackathon. If you want to use the results of our clustering, you can filter the dataset to be in the respective cluster. 


In [ ]:
clusters_df = pd.read_csv('data/Cluster_result.csv')
clusters_df.head()

In [ ]:
# Cluster of investors that are similar to Vanguard is cluster 1, and cluster of investors that are dissimilar to Vanguard is cluster 2
cluster_1 = clusters_df[clusters_df['kmeans_2']==0]
cluster_2 = clusters_df[clusters_df['kmeans_2']==1]
cluster_2.head()

## Investors' Info

We also provide you with more information for each investor in the holdings data.

The investor data we provide has the following columns:  
#### LGCYINVESTORID:
    The ID of the investor, for example , 2004260
    
#### Total Equity Assets: 
    Total equity assets managed by the investor on 2020-10-01. Note: we only have total equity assets on one specific date, 2020-10-01, so it's not reflecting the total equity assets of the investors on each HOLDDATE in the holdings dataframe
    
#### Turnover Rate Cal Time: 
    The date of Turnover Rate being calculated, it should match with HOLDDATE in holding dataframe

#### [Turnover Rate](https://news.morningstar.com/classroom2/course.asp?docId=2945&page=6&CN=COM): 
    Turnover rate of the investor

In [ ]:
investor_obj = s3.get_object(Bucket='ds-hackathon-data', Key='02-2021/investor.csv')
investor_df = pd.read_csv(investor_obj['Body'])

In [ ]:
investor_df.head()

## Instruments' Sectors

An investor's holding sometimes may lean towards a specific sector during a certain period, so it may focus on different underlying instrument but same sector, as you can see in this [news](https://www.fool.com/investing/2021/02/19/the-3-big-bank-moves-warren-buffett-made-last-quar/), Berkshire Hathaway, the company Warren Buffett works as CEO and chairman, has been reducing the holdings in banks(financials). So how can we track that? In this hackathon, We provide you with sector and industry information for each underlying instrument in the holdings data.

In this hackathon, we are using [The Refinitiv Business Classification](https://en.wikipedia.org/wiki/The_Refinitiv_Business_Classification) to seperate each underlying instrument into its Economic Sector, Business Sector, Industry Group, and Industry.

The sector data we provide has the following columns:  
#### instrument_ID: 
    The ID of the instrument, for example, 8590932301
    
#### EconDesc: 
    The economic sector of the instrument, for example, Technology

#### BusDesc:
    The business sector of the instrument, for example, Technology Equipment

#### GrpDesc: 
    The industry group of the instrument, for example, Computers, Phones & Household Electronics

#### IndDesc:
    The industry sector of the instrument, for example, Computer Hardware

In [ ]:
sector_path = 'data/instrument_sector.csv'
sector_df = pd.read_csv(sector_path)

In [ ]:
sector_df.head()

## Instrument Market Data

As stated above, the main question in this hackathon is 1. what investment insights/strategies can we glean from investors dissimilar to Vanguard 2. what investment conclusions can be drawn from those insights/strategies you found out. To help you better answer question 2, we are providing you market data for underlying instruments, you can match the instrument ID in market data with the holding dataframe.

But please note: 
1. It can be possible you run out of memory of the environment if you merge or join 2 large tables. In this scenario, please trim your tables by 
    1. Pre-select investors and/or instruments 
    2. Drop unnecessary columns in tables
2. The market data may <b>not be perfect</b>, which means it may contain errors or missing values. And since this is a real business problem, these things happen. Actually data being non-perfect almost happen in every real data science projects. The best things you can do are:
    1. Reseach(google) how to deal with partial wrong/missing data. Amend it, ignore it, not use the data? 
    2. Discuss in the teams channel with other people how to find a workaround if you encounter those problems in data.

The market data we provide has the following columns:  
#### MarketDate: 
    The data that the market data refers to
    
#### Open_: 
    The open price of the instrument

#### Close_:
    The close price of the instrument

#### INSTRID: 
    The ID of the instrument, for example, 8590932301

Due to the data size, we split the market data by year, you can load them as below:

In [ ]:
df_list= []
for year in range(16, 21):
    file_obj = s3.get_object(Bucket='ds-hackathon-data', Key=f'02-2021/market_20{str(year)}.csv')
    df_list.append(pd.read_csv(file_obj['Body']))
market_df = pd.concat(df_list)
market_df.head()

## ETF Market Data
After getting the sector information for each underlying instrument, we can get an idea about the composition of each investor's equity under management by grouping the data on the EconDesc field of each investor's holdings. 

How can we understand the performance of each sector? 
For this hackathon, we provide you with the [ETF](https://www.investopedia.com/terms/e/etf.asp) price which can be used to track the performance of the represented sector.

The [tickers](https://www.investopedia.com/ask/answers/12/what-is-a-stock-ticker.asp) of those ETFs are listed below.

In [ ]:
sector_etf_ticker = {'Financials': 'XLF',
 'Energy': 'XLE',
 'Industrials': 'DIA',
 'Healthcare': 'XLV',
 'Consumer Cyclicals': 'XLY',
 'Consumer Non-Cyclicals ': 'XLP',
 'Technology': 'XLK',
 'Basic Materials': 'GDX',
 'Real Estate': 'VNQ',
 'Utilities': 'XLU',
 'Telecommunications Services': 'XLC',
 'Academic & Educational Services': 'EDUT'}
sector_etf_ticker

The ETF price data we provide has the following columns:  
#### Ticker: 
    The ticker of the ETF which represents a sector
    
#### Date: 
    The date of the price 

#### Close:
    The close price of the ETF

In [ ]:
etf_price_path = 'data/etf_price.csv'
etf_price_df = pd.read_csv(etf_price_path)

In [ ]:
etf_price_df.head()

# Submission

Please prepare and send the requirements below to the shadi_fadaee@vanguard.com and chuqi_yang2@vanguard.com before deadline:
    
    1. A short summary of the insights you found.
    
    2. A presentation that contains the stories/insights/visualizations you generated which can be used to support your findings. Feel free to include anything else you would like to share with us, e.g. who you are, your workflow, any unsuccessful attempts and potential next steps.

# Awards

Raising the bar for Vanguard is an award for all of us. However, as this is an open-ended Hackathon and there will be <b>NO individual awards</b>. But please remember, this is a <b>vanguard business-related problem</b>, so we want to invite the teams who generated helpful insights to present their models and insights in the ceremony event; furthermore, if the result is insightful, we may work with you to refine the algorithm so that it can potentially be adopted by analysts in FIG! 

# Rules

    1. Do not download or dispense the data. 
    
    2. External data/packages are not allowed for safety reason. 
    
    3. Due to our capacity, the competition is only open to crews from IMG(including rotational crew) and IMG-related departments(PRD, GIFS, IMDP, TLP), please let us know if you are from other departments by emailing chuqi_yang2@vanguard.com or shadi_fadaee@vanguard.com.

    4. The hackathon is a skills-based session to promote and further the field of data science in investment. 

    5. Your competition submissions must conform to the requirements mentioned above.

    6. The hackathon period is from Feb 22rd - Mar 1st, any submission after Mar 1st 5:00 pm ET will not be considered. 

    7. We ask that you respect the spirit of the hackathon and not cheat. 

# Supplemental data

Due to the safety reason, please don't upload any data by yourself to the lab. If you want to add any data, please contact Chuqi Yang or Shadi Fadaee, we will be able to assist you on that